In [16]:
%%writefile mapper1.py

import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    pattern = re.compile('[A-Z][a-z0-9_-]+')
    counts = Counter(words)
    for word, word_count in counts.items():
        if pattern.match(word):
            print "%s\t%d\t%d" % (word.lower(), word_count, 1)
        else:
            print "%s\t%d\t%d" % (word.lower(), word_count, 0)
            
        

Overwriting mapper1.py


In [36]:
! cat mydata | python2 ./mapper1.py | python2 ./reducer1.py | head -10

key  anarcha-feminist 0 1
anarcha-feminist
key  writings 0 4
writings
key  dissolution 0 1
dissolution
key  four 0 1
four
key  prefix 0 1
prefix
Traceback (most recent call last):
  File "./reducer1.py", line 22, in <module>
    print "%s\t%d" % (current_key, name_words)
IOError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "./mapper1.py", line 21, in <module>
    print "%s\t%d\t%d" % (word.lower(), word_count, 0)
IOError: [Errno 32] Broken pipe
cat: write error: Broken pipe


In [37]:
%%writefile reducer1.py

import sys

def count_occurances(name_words, all_cases):
    return float(all_cases - name_words) / all_cases

current_key = None
name_words = 0
all_cases = 0
for line in sys.stdin:
    try:
        key, count, is_upper = line.strip().split('\t')
        count = int(count)
        is_upper = int(is_upper)
    except ValueError as e:
        continue
    
    if current_key != key:
        if current_key:
            print current_key
            if count_occurances(name_words, all_cases) < 0.005:
                print "%s\t%d" % (current_key, name_words)
        name_words = 0
        all_cases = 0
        current_key = key
    if is_upper:
        name_words += count
    all_cases += count

if current_key:
    if count_occurances(name_words, all_cases) < 0.005:
        print "%s\t%d" % (current_key, name_words)

Overwriting reducer1.py


In [10]:
%%writefile mapper2.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    
    print "%s\t%d" % (key, count)
        

Overwriting mapper2.py


In [38]:
%%bash

OUT_DIR1="wordcount_1_"$(date +"%s%6N")
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming NameCount 1" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR1} > /dev/null
    
OUT_DIR2="wordcount_2_"$(date +"%s%6N")
NUM_REDUCERS=1

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapred.text.key.comparator.options="-k2,2nr" \
    -D stream.num.map.output.key.fields=2 \
    -D mapred.jab.name="Streaming NameCount 2" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper2.py \
    -mapper "python mapper2.py" \
    -reducer "python mapper2.py" \
    -input ${OUT_DIR1} -output ${OUT_DIR2} > /dev/null
    
    
hdfs dfs -cat ${OUT_DIR2}/part-00000 | head -5 | tail -1


french	5741


rm: `wordcount_1_1549903931257430': No such file or directory
19/02/11 16:52:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/11 16:52:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/11 16:52:15 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/11 16:52:15 INFO mapreduce.JobSubmitter: number of splits:2
19/02/11 16:52:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549891060587_0047
19/02/11 16:52:15 INFO impl.YarnClientImpl: Submitted application application_1549891060587_0047
19/02/11 16:52:15 INFO mapreduce.Job: The url to track the job: http://9202b848d46a:8088/proxy/application_1549891060587_0047/
19/02/11 16:52:15 INFO mapreduce.Job: Running job: job_1549891060587_0047
19/02/11 16:52:20 INFO mapreduce.Job: Job job_1549891060587_0047 running in uber mode : false
19/02/11 16:52:20 INFO mapreduce.Job:  map 0% reduce 0%
19/02/11 16:52:36 INFO mapreduce.Job:  map 67% reduce 0%
19/02/11 16:52:39 